In [1]:
# !pip install ultralytics==8.0.196
# !pip install roboflow


In [14]:
from ultralytics import YOLO
import os
import cv2
import glob
import shutil
import matplotlib.pyplot as plt
import numpy as np
from roboflow import Roboflow
import torch
import json

In [ ]:
with open('config.json') as config_file:
    config = json.load(config_file)

# Access the API key
api_key = config.get('ROBOFLOW_API_KEY')

In [3]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("label-foor-images").project("cat_feeder_project")
version = project.version(8)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cat_feeder_project-8 in yolov8:: 100%|██████████| 13924/13924 [00:01<00:00, 8758.13it/s] 


# New section

In [4]:
# Define paths to the downloaded dataset
dataset_dir = dataset.location

train_img_dir = os.path.join(dataset_dir, 'train', 'images')
train_label_dir = os.path.join(dataset_dir, 'train', 'labels')
valid_img_dir = os.path.join(dataset_dir, 'valid', 'images')
valid_label_dir = os.path.join(dataset_dir, 'valid', 'labels')
test_img_dir = os.path.join(dataset_dir, 'test', 'images')
test_label_dir = os.path.join(dataset_dir, 'test', 'labels')


In [5]:
data_yaml_path = '/content/cat_feeder_project-8/data.yaml'

print(f"data_yaml_img_dir: {data_yaml_path}")

data_yaml_img_dir: /content/cat_feeder_project-8/data.yaml


In [6]:

model = YOLO("yolov8s.yaml")

model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=32,
    workers=4,
    )



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff85fad8eb0>
fitness: 0.8041329932552053
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.85864,     0.86592,     0.63456])
names: {0: 'Niuniu', 1: 'Orange', 2: 'human'}
plot: True
results_dict: {'metrics/precision(B)': 0.9484128284104801, 'metrics/recall(B)': 0.955356254520777, 'metrics/mAP50(B)': 0.9639625300457682, 'metrics/mAP50-95(B)': 0.7863741558340316, 'fitness': 0.8041329932552053}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.16746166868325135, 'inference': 2.836926613446962, 'loss': 0.0003183032152772156, 'postprocess': 2.2315838999904707}

In [7]:
metrics = model.val(data=data_yaml_path)


Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8s summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/cat_feeder_project-8/valid/labels.cache... 579 images, 0 backgrounds, 0 corrupt: 100%|██████████| 579/579 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/19 [00:00<?, ?it/s]Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:08<00:00,  2.34it/s]
                   all        579        621      0.948

In [ ]:
model.export()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (21.5 MB)

TorchScript: starting export with torch 2.3.0+cu121...
TorchScript: export success ✅ 1.8s, saved as 'runs/detect/train/weights/best.torchscript' (42.9 MB)

Export complete (3.2s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=640 data=/content/cat_feeder_project-6/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.torchscript'

In [ ]:
version.deploy("yolov8", "/content/runs/detect/train")

View the status of your deployment at: https://app.roboflow.com/label-foor-images/cat_feeder_project/8
Share your model with the world at: https://universe.roboflow.com/label-foor-images/cat_feeder_project/model/8


In [24]:
model = YOLO('/content/runs/detect/train/weights/best.pt')